# Логирование с ротацией файлов

- Периоды отсчитываются от момента запуска процесса логирования.
- Каждый период сохраняется в новый файл.
- Логи в реально времени пишутся в последний файл с логами.
- Название файлов задается по шаблону.
- Параметр **retention** задает число хранимых файлов логов или максимальное время их хранения.
- Параметр **compression** задает способ сжатия файлов с логами по завершенным периодам.

In [ ]:
import time

from loguru import logger

# Важная строчка, удаляет дефолтный логгер в консоль.
logger.remove()

log_handler = logger.add(

    # Шаблон названия файлов с логами.
    "app_{time:YYYY-MM-DD_HH-mm-ss}.log",

    # Параметр задает размер интервала ротации логов.
    rotation="1 minute",

    # Параметр задает максимальное время хранения логов.
    # Если в качестве значения установить целое число,
    # то удаляться будут самые старые файлы логов, всегда
    # будет оставаться не более указанного числа файлов.
    # Если в качестве значения установить строку с числом
    # и единицей измерения, то удаляться будут самые старые
    # файлы логов согласно заданному критерии времени.
    
    # retention="1 week",
    retention=2,

    # Если задать параметр compression, то при ротации
    # лог завершенного интервала будет сжат согласно
    # установленному значению параметра.
    
    # compression="zip",
)

def main():
    i = 0
    while True:
        i += 1
        time.sleep(1)
        logger.info(f"Пример лога: {i}")

try:
    main()
except KeyboardInterrupt:
    logger.remove(log_handler)

# Кастомный метод для определения границ ротации файлов

In [ ]:
import time
import datetime
import datetime

from loguru import logger

# Важная строчка, удаляет дефолтный логгер в консоль.
logger.remove()

class Rotator:

    def __init__(self, size, t):
        self._size = size

        # Если логирование должно ротироваться раз в сутки в определенный час, минуту, секунду.
        # self._time = datetime.datetime.now().replace(hour=t.hour, minute=t.minute, second=t.second)

        # Если логирование должно ротировать раз в час в определенную минуту и секунду.
        # self._time = datetime.datetime.now().replace(hour=t.hour, minute=t.minute, second=t.second)

        # Если логирование должно ротировать раз в минуту в определенную секунду.
        self._time = datetime.datetime.now().replace(second=t.second)

    def should_rotate(self, message, file):
        file.seek(0, 2)
        if file.tell() + len(message) > self._size:
            return True
        if message.record["time"].timestamp() > self._time.timestamp():
            self._time += datetime.timedelta(minutes=1)
            return True
        return False

rotator = Rotator(size=10_000, t=datetime.time(hour=12, minute=0, second=0))

log_handler = logger.add(
    "app_{time:YYYY-MM-DD_HH-mm-ss}.log",
    rotation=rotator.should_rotate,
    retention=2
)

def main():
    i = 0
    while True:
        i += 1
        time.sleep(1)
        logger.info(f"Пример лога: {i}")

try:
    main()
except KeyboardInterrupt:
    logger.remove(log_handler)

# Логирование разного уровня логов в разные файлы в разных форматах

In [ ]:
import sys
import time

from loguru import logger

# Важная строчка, удаляет дефолтный логгер в консоль.
logger.remove()

info_format = (
    "<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green>\t"
    "<level>{level: <3}</level>\t"
    "<yellow>{name}</yellow>:<cyan>{function}</cyan>:<magenta>{line}</magenta>\t"
    "<level>{message}</level>"
)

logger.add("log_info.txt", level="INFO", format=info_format)
logger.add("log_error.txt", level="ERROR", format=info_format)
logger.add(sys.stderr)

def main():
    i = 0
    while True:
        i += 1
        time.sleep(1)
        logger.info(f"Пример INFO лога: {i}")
        logger.error(f"Пример ERROR лога: {i}")

main()

# Добавление пользовательского уровня логирования

In [ ]:
import os
import sys
import time

from loguru import logger

# Важная строчка, удаляет дефолтный логгер в консоль.
logger.remove()

In [ ]:
log_level = os.getenv("LOG_LEVEL", "INFO").upper()

logger.level("CUSTOM", no=100, color="<magenta>", icon="💡")

def filter_custom(record):
    return record["level"].name != "CUSTOM"

logger.add("log_custom_level.txt", level="CUSTOM")
logger.add(sys.stderr, level=log_level, filter=filter_custom)


def main():
    i = 0
    while True:
        i += 1
        time.sleep(1)
        logger.info(f"Пример INFO лога: {i}")
        logger.error(f"Пример ERROR лога: {i}")
        logger.log("CUSTOM", f"Пример CUSTOM лога: {i}")

main()

# Логирование раздельно данных и сведений о работе с ротацией

In [ ]:
import os
import sys
import time
import datetime

from loguru import logger

# Важная строчка, удаляет дефолтный логгер в консоль.
logger.remove()

log_level = os.getenv("LOG_LEVEL", "DEBUG").upper()

logger.level("STORE_EVENT", no=100)

class Rotator:

    def __init__(self, t):
        self._time = datetime.datetime.now().replace(second=t.second)

    def should_rotate(self, message, _):
        if message.record["time"].timestamp() > self._time.timestamp():
            self._time += datetime.timedelta(minutes=1)
            return True
        return False
    
rotator = Rotator(t=datetime.time(second=0))

event_format = (
    "<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green>\t"
    "<level>{message}</level>"
)

log_format = (
    "<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green>\t"
    "<level>{level: <3}</level>\t"
    "<yellow>{name}</yellow>:<cyan>{function}</cyan>:<magenta>{line}</magenta>\t"
    "<level>{message}</level>"
)

logger.add(
    "events_{time:YYYY-MM-DD_HH-mm-ss}.csv",
    rotation=rotator.should_rotate,
    retention=2,
    level="STORE_EVENT",
    format=event_format,
)
logger.add(
    sys.stderr,
    level=log_level,
    filter=lambda record: record["level"].name != "STORE_EVENT",
    format=log_format,
)


def main():
    i = 0
    while True:
        i += 1
        time.sleep(1)
        logger.info(f"Итерация цикла: {i}")
        logger.log("STORE_EVENT", f"EVENT: {i}")
        logger.error(f"Ошибка в цикле: {i}")

main()